# STOCK FORECAST

In [ ]:
pip install numpy pandas matplotlib statsmodels


In [ ]:
!wget -O CK.zip "https://drive.google.com/uc?export=download&id=1bmWN7q5ic5dB1rgUYfXDEXJHi6QoY6KJ"
!unzip CK.zip

In [ ]:
# Đường dẫn đến thư mục dữ liệu
data_folder1 = "nyse"
data_folder2 = "nasdaq"
data_folder3 = "sp500"
data_folder4 = "forbes2000"

# Đọc dữ liệu giá cổ phiếu từ tệp CSV trong thư mục đã chọn
data = pd.read_csv(f"{data_folder1}/prices.csv")
data = pd.read_csv(f"{data_folder2}/prices.csv")
data = pd.read_csv(f"{data_folder3}/prices.csv")
data = pd.read_csv(f"{data_folder4}/prices.csv")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA


# Chuyển đổi cột ngày thành đối tượng datetime và đặt nó làm chỉ mục
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Xác định dữ liệu huấn luyện và dữ liệu kiểm tra
train_data = data['Close'][:-30]  # Sử dụng 30 ngày cuối cùng để kiểm tra
test_data = data['Close'][-30:]

# Xây dựng mô hình ARIMA
model = ARIMA(train_data, order=(1, 1, 1))
model_fit = model.fit()

# Dự đoán giá cổ phiếu trong tập kiểm tra
predictions = model_fit.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1, typ='levels')

# Hiển thị dự đoán và giá thực tế bằng đồ thị matplotlib
plt.figure(figsize=(12, 6))
plt.plot(data.index[-100:], data['Close'].values[-100:], color='blue', label='Giá thực tế')
plt.plot(predictions.index, predictions.values, color='red', label='Dự đoán')
plt.title('Dự đoán giá cổ phiếu bằng mô hình ARIMA')
plt.xlabel('Ngày')
plt.ylabel('Giá cổ phiếu')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Chuyển đổi cột ngày thành đối tượng datetime và đặt nó làm chỉ mục
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Lấy giá đóng cửa
close_prices = data['Close'].values

# Chuẩn hóa dữ liệu vào khoảng (0, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_prices.reshape(-1, 1))

# Tạo dữ liệu huấn luyện và kiểm tra
train_data = scaled_data[:-30]  # Sử dụng 30 ngày cuối cùng để kiểm tra
test_data = scaled_data[-30:]

# Chia dữ liệu thành các cửa sổ thời gian
window_size = 30

def create_sequences(data, window_size):
    x = []
    y = []
    for i in range(len(data)-window_size):
        x.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(x), np.array(y)

train_x, train_y = create_sequences(train_data, window_size)
test_x, test_y = create_sequences(test_data, window_size)

# Xây dựng mô hình LSTM
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(window_size, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Huấn luyện mô hình
model.fit(train_x, train_y, epochs=50, batch_size=16, verbose=1)

# Dự đoán giá cổ phiếu trong tập kiểm tra
predictions = model.predict(test_x)
predictions = scaler.inverse_transform(predictions)

# Hiển thị dự đoán và giá thực tế bằng đồ thị matplotlib
plt.figure(figsize=(12, 6))
plt.plot(data.index[-100:], data['Close'].values[-100:], color='blue', label='Giá thực tế')
plt.plot(data.index[-30:], predictions, color='red', label='Dự đoán')
plt.title('Dự đoán giá cổ phiếu bằng mô hình LSTM')
plt.xlabel('Ngày')
plt.ylabel('Giá cổ phiếu')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.transformer import Transformer

# Đọc dữ liệu giá cổ phiếu từ tệp CSV
data = pd.read_csv('stock_prices.csv')

# Chuyển đổi cột ngày thành đối tượng datetime và đặt nó làm chỉ mục
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Lấy giá đóng cửa
close_prices = data['Close'].values

# Chuẩn hóa dữ liệu vào khoảng (0, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_prices.reshape(-1, 1))

# Tạo dữ liệu huấn luyện và kiểm tra
train_data = scaled_data[:-30]  # Sử dụng 30 ngày cuối cùng để kiểm tra
test_data = scaled_data[-30:]

# Chia dữ liệu thành các cửa sổ thời gian
window_size = 30

def create_sequences(data, window_size):
    x = []
    y = []
    for i in range(len(data)-window_size):
        x.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(x), np.array(y)

train_x, train_y = create_sequences(train_data, window_size)
test_x, test_y = create_sequences(test_data, window_size)

# Xây dựng mô hình Transformer
input_seq = Input(shape=(window_size, 1))
transformer_block = Transformer(num_heads=2, ff_dim=32, num_transformer_blocks=1, d_model=16, dropout=0.2)(input_seq)
output = Dense(1)(transformer_block)
model = Model(inputs=input_seq, outputs=output)

# Compile và huấn luyện mô hình
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(train_x, train_y, epochs=50, batch_size=16, validation_split=0.1, callbacks=[early_stopping])

# Dự đoán giá cổ phiếu trong tập kiểm tra
predictions = model.predict(test_x)
predictions = scaler.inverse_transform(predictions)

# Hiển thị dự đoán và giá thực tế bằng đồ thị matplotlib
plt.figure(figsize=(12, 6))
plt.plot(data.index[-100:], data['Close'].values[-100:], color='blue', label='Giá thực tế')
plt.plot(data.index[-30:], predictions, color='red', label='Dự đoán')
plt.title('Dự đoán giá cổ phiếu bằng mô hình Transformer')
plt.xlabel('Ngày')
plt.ylabel('Giá cổ phiếu')
plt.legend()
plt.show()



---



# WEATHER FORECAST

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from google.colab import files

# Tải lên tệp "weatherHistory.csv"
uploaded = files.upload()

# Đọc dữ liệu thời tiết từ tệp CSV
data = pd.read_csv('weatherHistory.csv')

# Chuyển đổi cột ngày thành đối tượng datetime và đặt nó làm chỉ mục
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Xác định dữ liệu huấn luyện và dữ liệu kiểm tra
train_data = data['Temperature'][:-30]  # Sử dụng 30 ngày cuối cùng để kiểm tra
test_data = data['Temperature'][-30:]

# Xây dựng mô hình ARIMA
model = ARIMA(train_data, order=(1, 1, 1))
model_fit = model.fit()

# Dự đoán nhiệt độ trong tập kiểm tra
predictions = model_fit.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1, typ='levels')

# Hiển thị dự đoán và nhiệt độ thực tế bằng đồ thị matplotlib
plt.figure(figsize=(12, 6))
plt.plot(data.index[-100:], data['Temperature'].values[-100:], color='blue', label='Nhiệt độ thực tế')
plt.plot(predictions.index, predictions.values, color='red', label='Dự đoán')
plt.title('Dự đoán nhiệt độ bằng mô hình ARIMA')
plt.xlabel('Ngày')
plt.ylabel('Nhiệt độ')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from google.colab import files

# Đọc dữ liệu thời tiết từ tệp CSV
data = pd.read_csv('weatherHistory.csv')

# Chuyển đổi cột ngày thành đối tượng datetime và đặt nó làm chỉ mục
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Lấy nhiệt độ
temperature = data['Temperature (C)'].values

# Chuẩn hóa dữ liệu vào khoảng (0, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(temperature.reshape(-1, 1))

# Tạo dữ liệu huấn luyện và kiểm tra
train_data = scaled_data[:-30]  # Sử dụng 30 ngày cuối cùng để kiểm tra
test_data = scaled_data[-30:]

# Chia dữ liệu thành các cửa sổ thời gian
window_size = 30

def create_sequences(data, window_size):
    x = []
    y = []
    for i in range(len(data)-window_size):
        x.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(x), np.array(y)

train_x, train_y = create_sequences(train_data, window_size)
test_x, test_y = create_sequences(test_data, window_size)

# Xây dựng mô hình LSTM
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(window_size, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Huấn luyện mô hình
model.fit(train_x, train_y, epochs=50, batch_size=16, verbose=1)

# Dự đoán nhiệt độ trong tập kiểm tra
predictions = model.predict(test_x)
predictions = scaler.inverse_transform(predictions)

# Hiển thị dự đoán và nhiệt độ thực tế bằng đồ thị matplotlib
plt.figure(figsize=(12, 6))
plt.plot(data.index[-100:], temperature[-100:], color='blue', label='Nhiệt độ thực tế')
plt.plot(data.index[-30:], predictions, color='red', label='Dự đoán')
plt.title('Dự đoán nhiệt độ bằng mô hình LSTM')
plt.xlabel('Ngày')
plt.ylabel('Nhiệt độ')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.transformer import Transformer
from google.colab import files

# Đọc dữ liệu thời tiết từ tệp CSV
data = pd.read_csv('weatherHistory.csv')

# Chuyển đổi cột ngày thành đối tượng datetime và đặt nó làm chỉ mục
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Lấy nhiệt độ
temperature = data['Temperature (C)'].values

# Chuẩn hóa dữ liệu vào khoảng (0, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(temperature.reshape(-1, 1))

# Tạo dữ liệu huấn luyện và kiểm tra
train_data = scaled_data[:-30]  # Sử dụng 30 ngày cuối cùng để kiểm tra
test_data = scaled_data[-30:]

# Chia dữ liệu thành các cửa sổ thời gian
window_size = 30

def create_sequences(data, window_size):
    x = []
    y = []
    for i in range(len(data)-window_size):
        x.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(x), np.array(y)

train_x, train_y = create_sequences(train_data, window_size)
test_x, test_y = create_sequences(test_data, window_size)

# Xây dựng mô hình Transformer
input_seq = Input(shape=(window_size, 1))
transformer_block = Transformer(num_heads=2, ff_dim=32, num_transformer_blocks=1, d_model=16, dropout=0.2)(input_seq)
output = Dense(1)(transformer_block)
model = Model(inputs=input_seq, outputs=output)

# Compile và huấn luyện mô hình
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(train_x, train_y, epochs=50, batch_size=16, validation_split=0.1, callbacks=[early_stopping])

# Dự đoán nhiệt độ trong tập kiểm tra
predictions = model.predict(test_x)
predictions = scaler.inverse_transform(predictions)

# Hiển thị dự đoán và nhiệt độ thực tế bằng đồ thị matplotlib
plt.figure(figsize=(12, 6))
plt.plot(data.index[-100:], temperature[-100:], color='blue', label='Nhiệt độ thực tế')
plt.plot(data.index[-30:], predictions, color='red', label='Dự đoán')
plt.title('Dự đoán nhiệt độ bằng mô hình Transformer')
plt.xlabel('Ngày')
plt.ylabel('Nhiệt độ')
plt.legend()
plt.show()